In [119]:
import requests
from bs4 import BeautifulSoup
import csv
import webbrowser
import pandas as pd
import io
from reviews_final import scrape, write_in_csv

In [120]:
%run trip_advisor_final.py "2022/01/01" "2022/01/05" "recommended" "Barcelona"

Scraper Inititated for Locality:Barcelona
Finding search result page URL
URL found http://www.tripadvisor.com/Hotels-g187497-Barcelona_Catalonia-Hotels.html
http://www.tripadvisor.com/Hotels-g187497-Barcelona_Catalonia-Hotels.html
Parsing results 
http://www.tripadvisor.com/Hotels-g187497-oa30-Barcelona_Catalonia-Hotels.html
Parsing results 
http://www.tripadvisor.com/Hotels-g187497-oa60-Barcelona_Catalonia-Hotels.html
Parsing results 
Writing to output file tripadvisor_data.csv


TypeError: __init__() got an unexpected keyword argument 'headers'

In [ ]:
f

In [18]:
url = 'https://www.tripadvisor.com/Hotel_Review-g187497-d289670-Reviews-Gran_Hotel_Torre_Catalunya-Barcelona_Catalonia.html#REVIEWS'
lang = 'en'

In [40]:
def display(content, filename='output.html'):
    with open(filename, 'wb') as f:
        f.write(content)
        webbrowser.open(filename)

def get_soup(session, url, show=False):
    r = session.get(url)
    if show:
        display(r.content, 'temp.html')

    if r.status_code != 200: # not OK
        print('[get_soup] status code:', r.status_code)
    else:
        return BeautifulSoup(r.text, 'html.parser')

def post_soup(session, url, params, show=False):
    '''Read HTML from server and convert to Soup'''

    r = session.post(url, data=params)

    if show:
        display(r.content, 'temp.html')

    if r.status_code != 200: # not OK
        print('[post_soup] status code:', r.status_code)
    else:
        return BeautifulSoup(r.text, 'html.parser')

def scrapes(url, lang='ALL'):

    # create session to keep all cookies (etc.) between requests
    session = requests.Session()
    header = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.81 Safari/537.36'}
    #session = requests.Session(headers=header)

    session.headers.update({
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.81 Safari/537.36',
    })


    items = parse(session, url )

    return items

def parse(session, url):
    print('Get number of reviews and start getting subpages with reviews')

    print('[parse] url:', url)

    soup = get_soup(session, url)

    if not soup:
        print('[parse] no soup:', url)
        return

    #num_reviews = soup.find('span', class_='reviews_header_count').text # get text
    #num_reviews = num_reviews[1:-1]
    #num_reviews = num_reviews.replace(',', '')
    #num_reviews = int(num_reviews) # convert text into integer
    #print('[parse] num_reviews ALL:', num_reviews)

    url_template = url.replace('Reviews-', 'Reviews-or{}-')
    print('[parse] url_template:', url_template)

    items = []

    offset = 0

#https://www.tripadvisor.com/Hotel_Review-g187497-d289670-Reviews-Gran_Hotel_Torre_Catalunya-Barcelona_Catalonia.html#REVIEWS
#https://www.tripadvisor.com/Hotel_Review-g187497-d289670-Reviews-or5-Gran_Hotel_Torre_Catalunya-Barcelona_Catalonia.html#REVIEWS
#https://www.tripadvisor.com/Hotel_Review-g187497-d289670-Reviews-or10-Gran_Hotel_Torre_Catalunya-Barcelona_Catalonia.html#REVIEWS
#https://www.tripadvisor.com/Hotel_Review-g187497-d289670-Reviews-or15-Gran_Hotel_Torre_Catalunya-Barcelona_Catalonia.html#REVIEWS

    while(offset < 3):
        #print("offset"+offset)
        subpage_url = url_template.format(offset)

        subpage_items = parse_reviews(session, subpage_url)
        if not subpage_items:
            break

        items += subpage_items

        if len(subpage_items) < 3:
            break

        offset += 5

    return items

def get_reviews_ids(soup):

    items = soup.find_all('div', attrs={'data-reviewid': True})

    if items:
        reviews_ids = [x.attrs['data-reviewid'] for x in items][::2]
        print('[get_reviews_ids] data-reviewid:', reviews_ids)
        return reviews_ids

def get_more(session, reviews_ids):

    url = 'https://www.tripadvisor.com/OverlayWidgetAjax?Mode=EXPANDED_HOTEL_REVIEWS_RESP&metaReferer=Hotel_Review'

    payload = {
        'reviews': ','.join(reviews_ids), # ie. "577882734,577547902,577300887",
        #'contextChoice': 'DETAIL_HR', # ???
        'widgetChoice': 'EXPANDED_HOTEL_REVIEW_HSX', # ???
        'haveJses': 'earlyRequireDefine,amdearly,global_error,long_lived_global,apg-Hotel_Review,apg-Hotel_Review-in,bootstrap,desktop-rooms-guests-dust-en_US,responsive-calendar-templates-dust-en_US,taevents',
        'haveCsses': 'apg-Hotel_Review-in',
        'Action': 'install',
    }

    soup = post_soup(session, url, payload)

    return soup

def parse_reviews(session, url):
    '''Get all reviews from one page'''

    print('[parse_reviews] url:', url)

    soup =  get_soup(session, url)

    if not soup:
        print('[parse_reviews] no soup:', url)
        return

    hotel_name = soup.find('h1', id='HEADING').text

    reviews_ids = get_reviews_ids(soup)
    if not reviews_ids:
        return

    soup = get_more(session, reviews_ids)

    if not soup:
        print('[parse_reviews] no soup:', url)
        return

    items = []

    for idx, review in enumerate(soup.find_all('div', class_='reviewSelector')):

        badgets = review.find_all('span', class_='badgetext')
        if len(badgets) > 0:
            contributions = badgets[0].text
        else:
            contributions = '0'

        if len(badgets) > 1:
            helpful_vote = badgets[1].text
        else:
            helpful_vote = '0'
        user_loc = review.select_one('div.userLoc strong')
        if user_loc:
            user_loc = user_loc.text
        else:
            user_loc = ''

        bubble_rating = review.select_one('span.ui_bubble_rating')['class']
        bubble_rating = bubble_rating[1].split('_')[-1]

        item = {
            'review_body': review.find('p', class_='partial_entry').text,
            'review_date': review.find('span', class_='ratingDate')['title'], # 'ratingDate' instead of 'relativeDate'
        }

        items.append(item)
        print('\n--- review ---\n')
        for key,val in item.items():
            print(' ', key, ':', val)

    print()

    return items

def write_in_csv(items, filename='results.csv',
                  headers=['hotel name', 'review title', 'review body',
                           'review date', 'contributions', 'helpful vote',
                           'user name' , 'user location', 'rating'],
                  mode='w'):

    print('--- CSV ---')

    with io.open(filename, mode, encoding="utf-8") as csvfile:
        csv_file = csv.DictWriter(csvfile, headers)

        if mode == 'w':
            csv_file.writeheader()

        csv_file.writerows(items)


In [ ]:
items = scrapes(url, lang)

In [135]:
import requests             
from bs4 import BeautifulSoup 
import csv                  
import webbrowser
import io

def display(content, filename='output.html'):
    with open(filename, 'wb') as f:
        f.write(content)
        webbrowser.open(filename)

def get_soup(session, url, show=False):
    r = session.get(url)
    if show:
        display(r.content, 'temp.html')

    if r.status_code != 200: # not OK
        print('[get_soup] status code:', r.status_code)
    else:
        return BeautifulSoup(r.text, 'html.parser')
    
def post_soup(session, url, params, show=False):
    '''Read HTML from server and convert to Soup'''

    r = session.post(url, data=params)
    
    if show:
        display(r.content, 'temp.html')

    if r.status_code != 200: # not OK
        print('[post_soup] status code:', r.status_code)
    else:
        return BeautifulSoup(r.text, 'html.parser')
    
def scrape(url, lang='en'):

    # create session to keep all cookies (etc.) between requests
    session = requests.Session()

    session.headers.update({
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.81 Safari/537.36',
    })


    items = parse(session, url)

    return items

def parse(session, url):
    '''Get number of reviews and start getting subpages with reviews'''

    print('[parse] url:', url)

    soup = get_soup(session, url)

    if not soup:
        print('[parse] no soup:', url)
        return

    # num_reviews = soup.find('span', class_='reviews_header_count').text # get text
    # num_reviews = num_reviews[1:-1] 
    # num_reviews = num_reviews.replace(',', '')
    # num_reviews = int(num_reviews) # convert text into integer
    # print('[parse] num_reviews ALL:', num_reviews)

    url_template = url.replace('.html', '-or{}.html')
    print('[parse] url_template:', url_template)

    items = []

    offset = 0

    while(offset < 20):  #used to control the flow
        subpage_url = url_template.format(offset)

        subpage_items = parse_reviews(session, subpage_url)
        if not subpage_items:
            break

        items += subpage_items

        if len(subpage_items) < 5:
            break

        offset += 5
    print(f'some {items}')
    return items

def get_reviews_ids(soup):

    items = soup.find_all('div', attrs={'data-reviewid': True})

    if items:
        reviews_ids = [x.attrs['data-reviewid'] for x in items][::2]
        print('[get_reviews_ids] data-reviewid:', reviews_ids)
        return reviews_ids
    
def get_more(session, reviews_ids):
    print('we are i get more')
    url = 'https://www.tripadvisor.com/OverlayWidgetAjax?Mode=EXPANDED_HOTEL_REVIEWS_RESP&metaReferer=Hotel_Review'

    payload = {
        'reviews': ','.join(reviews_ids), # ie. "577882734,577547902,577300887",
        #'contextChoice': 'DETAIL_HR', # ???
        'widgetChoice': 'EXPANDED_HOTEL_REVIEW_HSX', # ???
        'haveJses': 'earlyRequireDefine,amdearly,global_error,long_lived_global,apg-Hotel_Review,apg-Hotel_Review-in,bootstrap,desktop-rooms-guests-dust-en_US,responsive-calendar-templates-dust-en_US,taevents',
        'haveCsses': 'apg-Hotel_Review-in',
        'Action': 'install',
    }

    soup = post_soup(session, url, payload)

    return soup

def parse_reviews(session, url):
    '''Get all reviews from one page'''

    print('[parse_reviews] url:', url)

    soup =  get_soup(session, url)

    if not soup:
        print('[parse_reviews] no soup:', url)
        return

    hotel_name = soup.find('h1', id='HEADING').text

    reviews_ids = get_reviews_ids(soup)
    if not reviews_ids:
        return

    # soup = get_more(session, reviews_ids)

    # if not soup:
    #     print('[parse_reviews] no soup:', url)
    #     return

    items = []
    dates = []
    itemss = []
    d = []
    r = []

    for idx, review in enumerate(soup.find_all('q')):


        item = {
            'review_body': review.span.text.strip(),
            
         #'review_date': review.find('span', class_='euPKI _R Me S4 H3')['title'] # <span class="euPKI _R Me S4 H3">Date of stay:</span> December 2021</span>
        }
        item_text = review.span.text.strip()
        items.append(item)
        r.append(item_text)

        print('\n--- review ---\n')
        for key,val in item.items():
            print(' ', key, ':', val)
        print()
    for idx, review in enumerate(soup.find_all("span",class_="euPKI _R Me S4 H3")):
        date={'reveiw_date':review.text[14:]}
        date_text = review.text[14:]
        d.append(date_text)
        dates.append(date)
    for i in range(len(d)):
        single = {
                'review_body': r[i],
                'review_date': d[i]

        }
        itemss.append(single)


    return itemss

def write_in_csv(items, filename='results.csv',
                  headers=['hotel name', 'review title', 'review body',
                           'review date', 'contributions', 'helpful vote',
                           'user name' , 'user location', 'rating'],
                  mode='w'):

    print('--- CSV ---')

    with io.open(filename, mode, encoding="utf-8") as csvfile:
        csv_file = csv.DictWriter(csvfile, headers)

        if mode == 'w':
            csv_file.writeheader()

        csv_file.writerows(items)
        
DB_COLUMN   = 'review_body'
DB_COLUMN1 = 'review_date'

start_urls = [
    'https://www.tripadvisor.ca/Hotel_Review-g60982-d87016-Reviews-Hilton_Hawaiian_Village_Waikiki_Beach_Resort-Honolulu_Oahu_Hawaii.html',
]

lang = 'en'

headers = [ 
    DB_COLUMN, 
    DB_COLUMN1, 
]

for url in start_urls:

    # get all reviews for 'url' and 'lang'
    items = scrape(url, lang)

    if not items:
        print('No reviews')
    else:
        # write in CSV
        filename = url.split('Reviews-')[1][:-5] + '__' + lang
        print('filename:', filename)
        write_in_csv(items, filename + '.csv', headers, mode='w')

[parse] url: https://www.tripadvisor.ca/Hotel_Review-g60982-d87016-Reviews-Hilton_Hawaiian_Village_Waikiki_Beach_Resort-Honolulu_Oahu_Hawaii.html
[parse] url_template: https://www.tripadvisor.ca/Hotel_Review-g60982-d87016-Reviews-Hilton_Hawaiian_Village_Waikiki_Beach_Resort-Honolulu_Oahu_Hawaii-or{}.html
[parse_reviews] url: https://www.tripadvisor.ca/Hotel_Review-g60982-d87016-Reviews-Hilton_Hawaiian_Village_Waikiki_Beach_Resort-Honolulu_Oahu_Hawaii-or0.html
[get_reviews_ids] data-reviewid: ['823119520', '823066006', '823007235']

--- review ---

  review_body : As the title says, old but beautiful area. Rooms are spacious for 2 double beds in waikiki. I just can’t understand why and how it takes so long to checkin. Everytime we come here, 45-60 minutes to checkin. This time we missed dinner reservation even though we arrived an hour and 15 minutes before thinking we’d have time as the dinner is in Waikiki too! Diamond line ends up taking longer than the regular line! Have Diamond, sa

In [126]:
import pandas as pd

In [136]:
df_r = pd.read_csv('Hilton_Hawaiian_Village_Waikiki_Beach_Resort-Honolulu_Oahu_Hawaii__en.csv')

In [137]:
df_r

,review_body,review_date
0,"As the title says, old but beautiful area. Roo...",December 2021
1,We stayed one night in the rainbow tower full ...,August 2021
2,The staff was friendly and helpful . Our Need...,December 2021
3,Madhouse over Christmas. Staff at Alii tower C...,December 2021
4,I’ve been to Hawaii 18 times since 2017 and st...,December 2021
5,"Resort was decent, rooms were clean, but other...",December 2021
6,"Ok hotel, not great. It took us 30 minutes to ...",December 2021
7,Had to same and my family rooms twice since fi...,December 2021
8,Being a diamond plus member I was provided a c...,December 2021
9,This was our second time staying at this resor...,December 2021
